# Design for Co-Factor Swapping

The co-factor swapping analysis can be used to find reactions where one cofactor can be switched for another, leading to increased availability for cofactors used in the target reaction, DHAA production.

The main function used in this analysis is CofactorSwapAnalysis. 

In order to enhance the availability of a cofactor that is rate-limiting for the generation of a target metabolite and raise its theoretical yield, CofactorSwapOptimization uses an algorithm to determine which reactions in the model can undergo a swap in cofactor specificity.


In [ ]:
# Initialising all required packages
from cobra.io import read_sbml_model, write_sbml_model #importing functions needed
from cobra import Reaction, Metabolite
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

In [ ]:
model = read_sbml_model("data/iYO844_modified.xml")
model

In [ ]:
# Calculating and printing the maximum theoretical growth rate and DHAA production using the current model
with model:
    base_max_growth = model.slim_optimize()
    print(base_max_growth)
    model.objective = model.reactions.ALDH1_CYP71AV1
    base_max_production = model.slim_optimize()
    print(base_max_production)

In [ ]:
# Setting up the mathematical programming solver for optimisation
# Free source GLPK (GNU Linear Programming Kit)
model.solver = "glpk" 

# Defining the biomass from the reaction describing biomass formation in the model
biomass = model.reactions.BIOMASS_BS_10 

# Setting lower bound for biomass
biomass.lower_bound = 0.1

# Defining the demand for target reaction dihydroartemisinic acid
demand = model.reactions.DM_DHAA_c

# Setting the objective of the model to the variable demand
model.objective = demand

In [ ]:
# Defining the product yield as DHAA production as a function of biomass
prod_yield = product_yield(model.reactions.DM_DHAA_c, biomass) 
prod_yield

In [ ]:
# Swapping NADPH to NADH                                                                               
swap_opt = CofactorSwapOptimization(model=model, objective_function = prod_yield, plot=True)

In [ ]:
%%time
result = swap_opt.run(max_size=5) #running the model, restricting to 5 possible co-factor swaps
result

The co-factor swap analysis reveals no reactions that, upon changing co-factor specificity, would lead to an increased theoretical yield of dihydroartemisinic acid.

### Manual cofactor swapping
Investigation of swapping the co-factor specificity of CYP71AV1 from NADP+NADH to NAD+NADH will allow an increased theoretical yield of dihydroartesiminic acid. Several of the introduced reactions utilize CYP71AV1; reaction 3, 4 and 6.

In [ ]:
# Generating a copy of iY0844_modified in a variable to add the reactions to the new model
iYO844_modified_swap = model.copy()

In [ ]:
# Creating the reaction
reaction_3_swap = Reaction('CYP71AV1_swap') # CYP71AV1: amorphadiene oxidase

# Adding the metabolites based on the stoichiometric reaction
reaction_3_swap.add_metabolites({model.metabolites.get_by_id('amorph_c'): -1,
                            model.metabolites.get_by_id('o2_c'): -1,
                            model.metabolites.get_by_id('nad_c'): -1,
                            model.metabolites.get_by_id('artalc_c'): 1, 
                            model.metabolites.get_by_id('nadh_c'): 1,
                            model.metabolites.get_by_id('h2o_c'): 1})

print(reaction_3_swap.build_reaction_string()) # Printing reaction to check
iYO844_modified_swap.add_reactions([reaction_3_swap]) # Adding the reaction to the new iY0844 model

In [ ]:
# Creating the reaction
reaction_6_swap = Reaction('ALDH1_CYP71AV1_swap') # ALDH1: aldehyde dehydrogenase 1, CYP71AV1: amorphadiene oxidase

# Adding the metabolites based on the stoichiometric reaction
reaction_6_swap.add_metabolites({model.metabolites.get_by_id('artalc_c'): -1,
                            model.metabolites.get_by_id('o2_c'): -1,
                            model.metabolites.get_by_id('nad_c'): -1,
                            model.metabolites.get_by_id('DHAA_c'): 1, 
                            model.metabolites.get_by_id('nadh_c'): 1,
                            model.metabolites.get_by_id('h2o_c'): 1})

print(reaction_6_swap.build_reaction_string()) # Printing reaction to check
iYO844_modified_swap.add_reactions([reaction_6_swap]) # Adding the reaction to the new iY0844 model

The original reactions 3, 4 and 6 are knocked out the new iYO844_modified_swap GSM

In [ ]:
model.reactions.CYP71AV1.knock_out()
model.reactions.ADH1_CYP71AV1.knock_out()
model.reactions.ALDH1_CYP71AV1.knock_out()

In [ ]:
# Calculating and printing the maximum theoretical growth rate and DHAA production using the coswap model
with iYO844_modified_swap:
    iYO844_modified_swap.objective = iYO844_modified_swap.reactions.BIOMASS_BS_10
    swap_max_growth = iYO844_modified_swap.slim_optimize()
    print(swap_max_growth)
    iYO844_modified_swap.objective = iYO844_modified_swap.reactions.ALDH1_CYP71AV1_swap
    swap_max_production = iYO844_modified_swap.slim_optimize()
    print(swap_max_production)

In [ ]:
print("Max growth has changed by", \
      round((swap_max_growth - base_max_growth) / base_max_growth * 100, 2), \
      "percent")

print("Max production has changed by", \
      round((swap_max_production - base_max_production) / base_max_production * 100, 2), \
      "percent")

Co-factor swapping of CYP71AV1 does not result in increased yield of DHAA. \

As a last resort, the co-factor swap analysis is run on this new swap model, to test if a model with two co-factor swaps (CYP71AV1 and another enzyme) would lead to increased yield. 

In [ ]:
# Swapping NADPH to NADH                                                                               
swap_opt = CofactorSwapOptimization(model=iYO844_modified_swap, objective_function = prod_yield, plot=True)

In [ ]:
%%time
result = swap_opt.run(max_size=5)
result

This second co-factor swap analysis also reveals no reactions that would lead to an increased theoretical yield of DHAA.